# AI video editor

### Setup



In [1]:
%tensorflow_version 2.x
!pip install -q sklearn
!pip install pyyaml h5py

from sklearn import model_selection
from sklearn.utils import shuffle

import os
import tensorflow as tf
import keras as keras
import pandas as pd
import matplotlib.pyplot as plt

## Data

### Data urls

In [4]:
data_urls = [
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_0.csv",
  # Not avialable "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_1.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_2.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_3.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_4.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_5.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_6.csv",
  # Not available "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_7.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_8.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_9.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_10.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_11.csv",
  # Not available "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_12.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_13.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_14.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_15.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_16.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_17.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_18.csv",
  # Not available "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_19.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_20.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_21.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_22.csv",
  "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_23.csv",
  # Not available "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_24.csv",
]

test_url = "https://raw.githubusercontent.com/EliasDolinsek/ai_video_cutter/master/data_collection/data/video_25.csv"

In [5]:
def get_data_from_urls(urls):
  data = []
  for data_url in urls:
    fetched_content = pd.read_csv(data_url, index_col=0, header=0)

    # remove timestamps
    del fetched_content["start_timestamp"]
    del fetched_content["end_timestamp"]

    # remove white sapces
    fetched_content.columns = fetched_content.columns.str.replace(' ', '')
    data.append(fetched_content)
  
  return data

### Data gathering

In [6]:
train_data = get_data_from_urls(data_urls)


In [7]:
test_data = get_data_from_urls([test_url])

In [8]:
def transform_to_data_list(data):
  return (dataset["cut"], dataset.drop("cut", axis=1))

#### Train data

In [9]:
y_train_data_list = []
x_train_data_list = []

for dataset in train_data:
  y,x = transform_to_data_list(dataset)
  y_train_data_list.append(y)
  x_train_data_list.append(x)

#### Test data

In [10]:
y_test_data_list = []
x_test_data_list = []

for dataset in test_data:
  y,x = transform_to_data_list(dataset)
  y_test_data_list.append(y)
  x_test_data_list.append(x)

## Model

### Create model

In [11]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(68,)),  # input layer (1)
    keras.layers.Dense(128, activation='relu'),  # hidden layer (2)
    keras.layers.Dense(2, activation='softmax') # output layer (3)
])

In [12]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 68)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               8832      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 9,090
Trainable params: 9,090
Non-trainable params: 0
_________________________________________________________________


### Train model

In [14]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
train_data_len = len(x_train_data_list)
for x_data, y_data, index in zip(x_train_data_list, y_train_data_list, range(train_data_len)):
  print(f"Starting fitting {index}/{train_data_len}")
  model.fit(x_data, y_data, epochs=100, callbacks=[cp_callback], verbose=0)
  print("\nFinished fitting\n")

Starting fitting 0/20

Epoch 00001: saving model to training_1/cp.ckpt

Epoch 00002: saving model to training_1/cp.ckpt

Epoch 00003: saving model to training_1/cp.ckpt

Epoch 00004: saving model to training_1/cp.ckpt

Epoch 00005: saving model to training_1/cp.ckpt

Epoch 00006: saving model to training_1/cp.ckpt

Epoch 00007: saving model to training_1/cp.ckpt

Epoch 00008: saving model to training_1/cp.ckpt

Epoch 00009: saving model to training_1/cp.ckpt

Epoch 00010: saving model to training_1/cp.ckpt

Epoch 00011: saving model to training_1/cp.ckpt

Epoch 00012: saving model to training_1/cp.ckpt

Epoch 00013: saving model to training_1/cp.ckpt

Epoch 00014: saving model to training_1/cp.ckpt

Epoch 00015: saving model to training_1/cp.ckpt

Epoch 00016: saving model to training_1/cp.ckpt

Epoch 00017: saving model to training_1/cp.ckpt

Epoch 00018: saving model to training_1/cp.ckpt

Epoch 00019: saving model to training_1/cp.ckpt

Epoch 00020: saving model to training_1/cp.ckp

### Save model

In [ ]:
model.save("ai_video_editor_model.h5")

### Download model

In [ ]:
from google.colab import files
files.download("ai_video_editor_model.h5")

### Load model

In [ ]:
model = tf.keras.models.load_model('ai_video_editor_model.h5')

### Test model

In [ ]:
for index, row in y_test_data_list[0].iteritems():
  if row != 1:
    y_test_data_list[0] = y_test_data_list[0].drop(index)
    x_test_data_list[0] = x_test_data_list[0].drop(index)

In [ ]:
test_data_len = len(x_test_data_list)
for x_data, y_data, index in zip(x_test_data_list, y_test_data_list, range(test_data_len)):
  test_loss, test_acc = model.evaluate(x_data, y_data, verbose=1) 
  print(f'Test accuracy of test data {index+1}/{test_data_len}:', test_acc)

2/2 [==============================] - 0s 2ms/step - loss: 112.3771 - accuracy: 1.0000
Test accuracy of test data 1/1: 1.0


### Predict

In [ ]:
x_test_data_list[0].iloc[[0]]

,zcr,energy,energy_entropy,spectral_centroid,spectral_spread,spectral_entropy,spectral_flux,spectral_rolloff,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,chroma_1,chroma_2,chroma_3,chroma_4,chroma_5,chroma_6,chroma_7,chroma_8,chroma_9,chroma_10,chroma_11,chroma_12,chroma_std,deltazcr,deltaenergy,deltaenergy_entropy,deltaspectral_centroid,deltaspectral_spread,deltaspectral_entropy,deltaspectral_flux,deltaspectral_rolloff,deltamfcc_1,deltamfcc_2,deltamfcc_3,deltamfcc_4,deltamfcc_5,deltamfcc_6,deltamfcc_7,deltamfcc_8,deltamfcc_9,deltamfcc_10,deltamfcc_11,deltamfcc_12,deltamfcc_13,deltachroma_1,deltachroma_2,deltachroma_3,deltachroma_4,deltachroma_5,deltachroma_6,deltachroma_7,deltachroma_8,deltachroma_9,deltachroma_10,deltachroma_11,deltachroma_12,deltachroma_std
155,0.000394,0.000394,0.000401,0.000415,0.000336,0.000338,0.000259,0.00025,0.000177,0.00016,0.000132,0.000134,0.00104,0.00276,0.002153,0.002367,0.003731,0.002452,0.00213,0.002104,0.001542,0.00302,0.003379,0.004443,0.005644,0.002782,0.001432,0.003413,0.003163,0.002347,0.002219,0.002012,0.002199,0.001287,0.001076,0.001398,0.001511,0.00239,0.002355,0.002018,0.001992,0.001337,0.002303,0.003019,0.002174,0.002235,0.002295,0.001766,0.001869,0.001957,0.001724,0.001261,0.001426,0.002064,0.00153,0.001088,0.001271,0.001102,0.00094,0.000737,0.000973,0.001019,0.000479,0.000452,0.000511,0.000497,0.000969,0.001033


In [ ]:
predictions = model.predict(x_test_data_list[0].iloc[[5]])
print(predictions)

[[9.9981600e-01 1.8396918e-04]]
